In [2]:
import os
import sys
import pickle
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, auc
import seaborn as sns
import json
import pathlib
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostClassifier, CatBoostRegressor
from xgboost import XGBClassifier, XGBRegressor
import copy
import json
import pandas as pd
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, r2_score
from scipy.stats import pearsonr
from sklearn.metrics import f1_score, make_scorer, matthews_corrcoef

In [8]:
# Загрузка данных
# ppi_data = pd.read_csv('PPI_features.csv')
abag_data = pd.read_csv('../feature_engineering/features/AbAg_features_reduced.csv')

,ID,Hyd_Hyd_4,Hyd_Pos_4,Hyd_Neg_4,Hyd_Acc_4,Hyd_Don_4,Hyd_Aro_4,Hyd_Sul_4,Hyd_Neu_4,Pos_Hyd_4,...,∆AAvolume,∆AAhydropathy,∆AAarea,∆AAweight,∆AAcharge,∆AAflexibily,∆AAchemical,∆AAsize,∆AAhbonds,ddG
0,1,91.0,6,6,45,52,85,1,64,6,...,-22.5,5.3,-35.0,-44.010,1.0,-17.0,-2.0,-1.0,-3.0,0.48
1,2,89.0,9,16,64,57,69,0,94,9,...,-78.1,-2.0,-55.0,-42.081,0.0,-8.0,0.0,-3.0,0.0,-0.99
2,3,57.0,12,7,63,60,13,0,101,12,...,-25.5,5.3,-45.0,-43.025,0.0,-35.0,-3.0,-1.0,-2.0,-1.08
3,4,201.0,16,9,97,103,246,0,116,16,...,-27.5,2.5,-25.0,-30.026,0.0,-2.0,-5.0,-1.0,-2.0,0.07
4,5,142.0,15,4,100,94,173,0,101,15,...,-27.5,2.5,-25.0,-30.026,0.0,-2.0,-5.0,-1.0,-2.0,-1.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,1806,161.0,0,6,66,76,224,0,96,0,...,27.5,-2.5,25.0,30.026,-0.0,2.0,5.0,1.0,2.0,-0.34
1796,1807,184.0,2,7,72,73,255,0,122,2,...,105.0,-3.1,115.0,92.097,-0.0,17.0,6.0,4.0,2.0,-3.21
1797,1808,164.0,0,5,60,72,221,1,97,0,...,105.0,-3.1,115.0,92.097,-0.0,17.0,6.0,4.0,2.0,-0.36
1798,1809,306.0,0,6,95,115,483,5,122,0,...,105.0,-3.1,115.0,92.097,-0.0,17.0,6.0,4.0,2.0,-1.72


In [5]:
AbRFR = RandomForestRegressor(random_state=42)
AbRFC = RandomForestClassifier(random_state=42)
AbETR = ExtraTreesRegressor(random_state=42)
AbETC = ExtraTreesClassifier(random_state=42)
AbBR = XGBRegressor(random_state=42)
AbBC = XGBClassifier(random_state=42)

In [62]:
# Функция для вычисления корреляции Пирсона
def pearson_correlation(y_true, y_pred):
    correlation, _ = pearsonr(y_true, y_pred)
    return correlation
pearson_scorer = make_scorer(pearson_correlation, greater_is_better=True)

In [2]:
# Использование индексов для валидации из статьи
with open('CVIDS.json', 'r') as file:
    cvids = json.load(file)

# Преобразование индексов в список для каждого фолда
cv_indices = []
for train_indices, test_indices in [cvids[str(i)] for i in range(5)]:
    train_indices = list(map(int, train_indices))
    test_indices = list(map(int, test_indices))
    cv_indices.append([train_indices, test_indices])

# Список чисел, которые нужно пропустить, мы их удалили из датасета (в них цистеин мутировал)
skip_numbers = [461, 462, 463, 631, 632, 1366, 1367, 1368, 1536, 1537]

# Функция для перенумерации чисел
def renumber_lists(lists, skip_numbers):
    for sublist in lists:
        for inner_list in sublist:
            for i in range(len(inner_list)):
                # Находим количество чисел в skip_numbers, которые меньше текущего числа
                skip_count = sum(1 for num in skip_numbers if num < inner_list[i])
                # Уменьшаем число на количество пропущенных чисел
                inner_list[i] -= skip_count
                # Уменьшаем число на 1, чтобы начать нумерацию с 0
                inner_list[i] -= 1
    return lists

cv_indices = renumber_lists(cv_indices, skip_numbers)

In [60]:
def perform_custom_cv_reg(model, features):
    y = features['ddG']
    X = features.drop([], axis=1)

    # Функция для вычисления корреляции Пирсона
    def pearson_correlation(y_true, y_pred):
        correlation, _ = pearsonr(y_true, y_pred)
        return correlation
    # Создаем скорер для корреляции Пирсона
    pearson_scorer = make_scorer(pearson_correlation, greater_is_better=True)

    # Определение метрик: хотим r2, rmse и коэффициент Пирсона смотреть
    scorers = {
        'r2': make_scorer(r2_score),
        'rmse': make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False),
        'pearson': pearson_scorer
    }

    # Выполнение кросс-валидации для каждой метрики
    for name, scorer in scorers.items():
        scores = cross_val_score(model, X, y, cv=cv_indices, scoring=scorer, n_jobs=-1)
        print(f"Среднее значение {name}: {scores.mean()}")

In [61]:
def perform_custom_cv_clf(model, features):
    features['class'] = features['ddG'].apply(lambda x: 0 if x > -0.21 else 1) # 0 - безвредная, 1 - вредная
    y = features['class']
    X = features.drop(['ddG', 'class'], axis=1)

    # Определение метрик: хотим F1-measure и MCC
    scorers = {
        'f1': make_scorer(f1_score, average='binary'),
        'mcc': make_scorer(matthews_corrcoef)
    }

    # Выполнение кросс-валидации для каждой метрики
    for name, scorer in scorers.items():
        scores = cross_val_score(model, X, y, cv=cv_indices, scoring=scorer, n_jobs=-1)
        print(f"Среднее значение {name}: {scores.mean()}")


In [100]:
# GridSearchCV — 'n_estimators': [200], 'max_depth': [None, 10, 20, 30, 40, 50]
with open('CVIDS.json', 'r') as file:
    cvids = json.load(file)

# Преобразование индексов в список для каждого фолда
cv_indices = []
for train_indices, test_indices in [cvids[str(i)] for i in range(5)]:
    train_indices = list(map(int, train_indices))
    test_indices = list(map(int, test_indices))
    cv_indices.append([train_indices, test_indices])

cv_indices = renumber_lists(cv_indices, skip_numbers)

features = pd.read_csv('../feature_engineering/features/AbAg_features_1800.csv', index_col='ID')
# features = pd.read_csv('./AbRFC/data/train/Xtrain.csv')


# Определение параметров для поиска
param_grid = {
    'n_estimators': [200], #[100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'bootstrap': [True, False]
}



# Инициализация GridSearchCV с разбиением из статьи
grid_search = GridSearchCV(estimator=AbRFR, param_grid=param_grid, cv=cv_indices, scoring=pearson_scorer, n_jobs=-1) # scoring='neg_root_mean_squared_error'


grid_search.fit(X, y)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Лучшие параметры: {best_params}")
print(f"Лучшая оценка: {best_score}")


Лучшие параметры: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Лучшая оценка: 0.995498770577407


In [105]:
# GridSearchCV — 'n_estimators': [200], 'max_depth': [None, 10, 20, 30, 40, 50]

# Определение параметров для поиска
param_grid = {
    'n_estimators': [200], #[100, 200, 300, 400, 500],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'bootstrap': [True, False]
}

AbRFR = RandomForestRegressor(random_state=42)

# Инициализация GridSearchCV с разбиением из статьи
grid_search = GridSearchCV(estimator=AbRFR, param_grid=param_grid, cv=cv_indices, scoring=pearson_scorer, n_jobs=-1) # scoring='neg_root_mean_squared_error'


# Поиск оптимальных гиперпараметров
y = features['ddG']
X = features.drop([], axis=1)
grid_search.fit(X, y)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Лучшие параметры: {best_params}")
print(f"Лучшая оценка: {best_score}")


Лучшие параметры: {'bootstrap': True, 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Лучшая оценка: 0.995498770577407


### Тестирую модель с параметрами из статьи

In [5]:
# Задаем параметры RF из статьи
clf_params = {"bootstrap": True, "class_weight": None, "criterion": "entropy", "max_depth": 50, "max_features": "sqrt", "max_leaf_nodes": None, "min_impurity_decrease": 0.0, "min_samples_leaf": 10, "min_samples_split": 10, "min_weight_fraction_leaf": 0.0, "n_estimators": 1000, "n_jobs": None, "oob_score": False, "random_state": None, "verbose": 0, "warm_start": False}
reg_params = copy.deepcopy(clf_params)
del reg_params['class_weight']
del reg_params['criterion']

In [6]:
# AbRFR AbAg_features
features = pd.read_csv('../feature_engineering/features/AbAg_features.csv')
AbRFR_from_article = RandomForestRegressor(**reg_params)
perform_custom_cv_reg(AbRFR_from_article, features)

Среднее значение r2: 0.36450824226930106
Среднее значение rmse: -1.1547691243422722
Среднее значение pearson: 0.6795349294842672


In [119]:
# AbRFR AbAg_features_reduced
features = pd.read_csv('../feature_engineering/features/AbAg_features_reduced.csv')
AbRFR_from_article = RandomForestRegressor(**reg_params)
perform_custom_cv_reg(AbRFR_from_article, features)

Среднее значение r2: 0.377381095247839
Среднее значение rmse: -1.1491978792112456
Среднее значение pearson: 0.6862666120293615


In [11]:
# AbRFC AbAg_features
features = pd.read_csv('../feature_engineering/features/AbAg_features.csv')
AbRFC_from_article = RandomForestClassifier(**clf_params)
perform_custom_cv_clf(AbRFC_from_article, features)

Среднее значение f1: 0.7858924354914489
Среднее значение mcc: 0.2859244718240997


In [10]:
# AbRFC AbAg_features_reduced
features = pd.read_csv('../feature_engineering/features/AbAg_features_reduced.csv')
AbRFC_from_article = RandomForestClassifier(**clf_params)
perform_custom_cv_clf(AbRFC_from_article, features)


Среднее значение f1: 0.7879904985778998
Среднее значение mcc: 0.2798567269422732


In [13]:
# ExtraTreesRegressor AbAg_features_reduced
features = pd.read_csv('../feature_engineering/features/AbAg_features_reduced.csv')
ETR_from_article = ExtraTreesRegressor(**reg_params)
perform_custom_cv_reg(ETR_from_article, features)

Среднее значение r2: 0.2160245226837379
Среднее значение rmse: -1.284508826323058
Среднее значение pearson: 0.5350816327314425


In [48]:
# ExtraTreesRegressor AbAg_features
features = pd.read_csv('../feature_engineering/features/AbAg_features.csv')
ETR_from_article = ExtraTreesRegressor(**reg_params)
perform_custom_cv_reg(ETR_from_article, features)

Среднее значение r2: 0.19446622472869296
Среднее значение rmse: -1.301551331691376
Среднее значение pearson: 0.5205376595042515


### Пробую применить StandardScaler

In [36]:
features = pd.read_csv('../feature_engineering/features/AbAg_features_reduced.csv')
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
features_scaled = pd.DataFrame(features_scaled, columns=features.columns)

In [55]:
# AbRFR AbAg_features_reduced
AbRFR_from_article = RandomForestRegressor(**reg_params)
perform_custom_cv_reg(AbRFR_from_article, features_scaled)

Среднее значение r2: 0.38838754372803663
Среднее значение rmse: -0.7641446913920482
Среднее значение pearson: 0.6902852527553923


In [57]:
# AbRFC features_scaled
AbRFC_from_article = RandomForestClassifier(**clf_params)
perform_custom_cv_clf(AbRFC_from_article, features_scaled)

Среднее значение f1: 0.26130997060376765
Среднее значение mcc: 0.15085951716031404
